In [ ]:
# %pip install langchain
# %pip install huggingface_hub
# %pip install sentence_transformers
# %pip install faiss-cpu
# %pip install unstructured
# %pip install chromadb
# %pip install Cython
# %pip install tiktoken
# %pip install unstructured[local-inference]
# %pip install qdrant-client

In [2]:
import os
import requests

from langchain.embeddings import HuggingFaceEmbeddings #for using HugginFace models
# Vectorstore: https://python.langchain.com/en/latest/modules/indexes/vectorstores.html
from langchain.vectorstores import FAISS  #facebook vectorizationfrom langchain.chains.question_answering import load_qa_chain
from langchain.chains.question_answering import load_qa_chain
from langchain.document_loaders import DirectoryLoader
from langchain.document_loaders import TextLoader
from langchain.embeddings import HuggingFaceEmbeddings

from LangChain_chatbot_util import read_api_key

from langchain.text_splitter import CharacterTextSplitter

os.environ["HUGGINGFACEHUB_API_TOKEN"] = read_api_key()

In [3]:
loader = DirectoryLoader('./contexts', glob="./*.txt")
 
data = loader.load()

Extension horovod.torch has not been built: /usr/local/lib/python3.10/dist-packages/horovod/torch/mpi_lib_v2.cpython-310-x86_64-linux-gnu.so not found
If this is not expected, reinstall Horovod with HOROVOD_WITH_PYTORCH=1 to debug the build error.
Warning! MPI libs are missing, but python applications are still available.


In [8]:
print(data[0].page_content)

Autism spectrum disorder (ASD) is a developmental disability caused by differences in the brain. Some people with ASD have a known difference, such as a genetic condition. Other causes are not yet known. Scientists believe there are multiple causes of ASD that act together to change the most common ways people develop. We still have much to learn about these causes and how they impact people with ASD.

People with ASD may behave, communicate, interact, and learn in ways that are different from most other people. There is often nothing about how they look that sets them apart from other people. The abilities of people with ASD can vary significantly. For example, some people with ASD may have advanced conversation skills whereas others may be nonverbal. Some people with ASD need a lot of help in their daily lives; others can work and live with little to no support.

ASD begins before the age of 3 years and can last throughout a person’s life, although symptoms may improve over time. Som

In [9]:
text_splitter = CharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=0
)

docs_split = text_splitter.split_documents(data)
docs_split

[Document(page_content='Autism spectrum disorder (ASD) is a developmental disability caused by differences in the brain. Some people with ASD have a known difference, such as a genetic condition. Other causes are not yet known. Scientists believe there are multiple causes of ASD that act together to change the most common ways people develop. We still have much to learn about these causes and how they impact people with ASD.\n\nPeople with ASD may behave, communicate, interact, and learn in ways that are different from most other people. There is often nothing about how they look that sets them apart from other people. The abilities of people with ASD can vary significantly. For example, some people with ASD may have advanced conversation skills whereas others may be nonverbal. Some people with ASD need a lot of help in their daily lives; others can work and live with little to no support.', metadata={'source': 'contexts/ASD_general.txt'}),
 Document(page_content='ASD begins before the

In [10]:
# Embeddings
embeddings = HuggingFaceEmbeddings()

In [11]:
db = FAISS.from_documents(docs_split, embeddings)
query = "My teenager had a screening for autism. Where can I go for depression therapy services? "
docs = db.similarity_search(query)

docs

# for i in range(4):
#     print("\n")
#     print(docs[i].page_content)

[Document(page_content='What Is Autism Screening?\n\nAutism screening refers to the process of identifying individuals who may be at risk for autism spectrum disorder (ASD) or who may exhibit early signs of the condition. The primary goal of screening is to identify individuals who may benefit from further, more in-depth assessments or evaluations.\n\nAutism screening is usually administered by a pediatrician during a routine well-child checkup. The doctor may observe the child or ask you questions about his or her behaviors at home and while interacting with other children on the playground. The purpose of an autism screening is to identify common early signs of autism. The following are early signs of autism, however, they do not necessarily need to be present.\n\nDelays in communicating with others including speech and gestures\n\nNot responding to one’s own name\n\nGetting upset over minor changes in routine\n\nAvoiding many, if not most, forms of physical contact\n\nShowing little